In [8]:
!pip install ultralytics==8.1.0
!pip install pandas numpy matplotlib seaborn scikit-learn pingouin

In [9]:
import itertools
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [10]:
!rm -rf Deep_learning
!git clone https://github.com/lisapraino/Deep_learning.git

Cloning into 'Deep_learning'...
remote: Enumerating objects: 5031, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 5031 (delta 6), reused 18 (delta 5), pack-reused 5009 (from 2)
Receiving objects: 100% (5031/5031), 371.39 MiB | 17.06 MiB/s, done.
Resolving deltas: 100% (10/10), done.
Updating files: 100% (6301/6301), done.


In [11]:
yaml_path = "/content/Deep_learning/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml"

if not os.path.exists(yaml_path):
    raise FileNotFoundError("File data.yaml not found.")
else:
    print("data.yaml found :", yaml_path)

data.yaml found : /content/Deep_learning/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml


In [12]:
!pip install ultralytics --upgrade

  Using cached ultralytics-8.3.228-py3-none-any.whl.metadata (37 kB)
Using cached ultralytics-8.3.228-py3-none-any.whl (1.1 MB)
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.1.0
    Uninstalling ultralytics-8.1.0:
      Successfully uninstalled ultralytics-8.1.0


In [13]:
from ultralytics import YOLO

MODEL_PATH = "yolo11s"

base_model = YOLO(MODEL_PATH)

In [14]:
BATCH_SIZES = [8, 16, 32]
LEARNING_RATES = [1e-3, 1e-4, 1e-5]
EPOCHS = [25, 50, 100]

N_REPETITIONS = 5

In [15]:
factorial_combinations = list(itertools.product(BATCH_SIZES, LEARNING_RATES, EPOCHS))

print(f"Total number of combinations : {len(factorial_combinations)}")
print(f"Total number of runs : {len(factorial_combinations) * N_REPETITIONS}")

Total number of combinations : 27
Total number of runs : 135


In [16]:
def run_experiment(batch_size, learning_rate, epochs, rep):

    model = YOLO(MODEL_PATH)

    start = time.time()

    train_results = model.train(
        data=yaml_path,
        epochs=epochs,
        batch=batch_size,
        lr0=learning_rate,
        imgsz=640,
        device=0
    )

    train_time = time.time() - start

    metrics = model.val()

    return {
        "batch_size": batch_size,
        "learning_rate": learning_rate,
        "epochs": epochs,
        "rep": rep,
        "mAP": metrics.box.map,
        "precision": metrics.box.mp,
        "recall": metrics.box.mr,
        "f1": metrics.box.f1,
        "train_time": train_time,
        "train_loss": train_results.results_dict.get("train/box_loss", None),
        "val_loss": train_results.results_dict.get("val/box_loss", None)
    }

In [ ]:
all_results = []

for batch_size, lr, epochs in factorial_combinations:
    for rep in range(1, N_REPETITIONS + 1):

        print(f"\n➡ RUN — Batch={batch_size}, LR={lr}, Epochs={epochs}, Rep={rep}")

        result = run_experiment(batch_size, lr, epochs, rep)
        all_results.append(result)

        pd.DataFrame(all_results).to_csv("results_autosave.csv", index=False)


➡ RUN — Batch=8, LR=0.001, Epochs=25, Rep=1
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Deep_learning/alternative-dataset/Fridge_detection.v1i.yolov11/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train4, nbs=64, nms=False, opset=None, opti

In [ ]:
df = pd.DataFrame(all_results)
df.to_csv("results_final.csv", index=False)
df.head()

In [ ]:
sns.boxplot(data=df, x="batch_size", y="mAP")
plt.title("mAP vs Batch Size")
plt.show()

sns.boxplot(data=df, x="learning_rate", y="mAP")
plt.title("mAP vs Learning Rate")
plt.show()

sns.boxplot(data=df, x="epochs", y="mAP")
plt.title("mAP vs Epochs")
plt.show()

In [ ]:
import pingouin as pg

anova = pg.anova(
    data=df,
    dv="mAP",
    between=["batch_size", "learning_rate", "epochs"],
    detailed=True
)
anova

In [ ]:
import scipy.stats as stats

stats.probplot(df["mAP"], dist="norm", plot=plt)
plt.title("Normal Probability Plot")
plt.show()

In [ ]:
from scipy.stats import levene

levene(
    df[df["batch_size"] == 8]["mAP"],
    df[df["batch_size"] == 16]["mAP"],
    df[df["batch_size"] == 32]["mAP"]
)

In [ ]:
best = df.sort_values(by="mAP", ascending=False).iloc[0]
best